## CH 03 - Creating and Persisting DataFrames

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 10, 'display.max_rows', 10)

### How to do it\...

In [ ]:
# Usually, we create a DataFrame from an existing file or a database, but we can also create
# one from scratch. We can create a DataFrame from parallel lists of data.

In [6]:
fname = ['Paul', 'John', 'Richard', 'George']
lname = ['McCartney', 'Lennon', 'Starkey', 'Harrison']
birth = [1942, 1940, 1940, 1943]

In [7]:
# Create a dictionary from the lists, mapping the column name to the list:

people = {'first': fname, 'last': lname, 'birth': birth}

In [8]:
# Create a DataFrame from the dictionary:

beatles = pd.DataFrame(people)
beatles

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


### How it works\...

In [ ]:
# By default, pandas will create a RangeIndex for our DataFrame when we call the constructor:

beatles.index

In [ ]:
# We can specify another index for the DataFrame if we desire:

pd.DataFrame(people, index=['a', 'b', 'c', 'd'])

### There\'s More

In [ ]:
# You can also create a DataFrame from a list of dictionaries:

pd.DataFrame(
[{"first":"Paul","last":"McCartney", "birth":1942},
 {"first":"John","last":"Lennon", "birth":1940},
 {"first":"Richard","last":"Starkey", "birth":1940},
 {"first":"George","last":"Harrison", "birth":1943}])

### Writing CSV\...

In [ ]:
beatles

In [ ]:
# Write the DataFrame to a CSV file

from io import StringIO
fout = StringIO()
beatles.to_csv(fout)  # use a filename instead of fout

In [ ]:
print(fout.getvalue())

### There\'s More

In [ ]:
# The .to_csv method has a few options. You will notice that it included the index in the
# output but did not give the index a column name. If you were to read this CSV file into
# a DataFrame using the read_csv function, it would not use this as the index by default.

# Instead, you will get a column named Unnamed: 0 in addition to an index. These columns
# are redundant:

_ = fout.seek(0)
pd.read_csv(fout)

In [ ]:
# fout: It represents a file on the filesystem.

# seek(0): The seek method is used to change the current file position. In this case, seek(0) 
# is setting the file pointer to the beginning of the file (offset 0).

# _ = : The underscore _ is a convention in Python often used as a throwaway variable name when 
# the value of the variable is not going to be used. It indicates that the result of the seek(0) 
# operation is being ignored or not explicitly used in the code.

# So, the overall effect of this line of code is to move the file pointer to the beginning of the 
# file represented by the fout object. This can be useful, for example, when you want to read the 
# contents of the file again from the start or overwrite its contents.

In [ ]:
# The read_csv function has an index_col parameter that you can use to specify the
# location of the index:

_ = fout.seek(0)
pd.read_csv(fout, index_col=0)

In [ ]:
# Alternatively, if we didn't want to include the index when writing the CSV file, we can set the
# index parameter to False:

fout = StringIO()
beatles.to_csv(fout, index=False) 
print(fout.getvalue())

In [ ]:
# The pandas library is an in-memory tool. You need to be able to fit your data in memory to use
# pandas with it. If you come across a large CSV file that you want to process, you have a few
# options. 

# If you can process portions of it at a time, you can read it into chunks and process
# each chunk. Alternatively, if you know that you should have enough memory to load the file,
# there are a few hints to help pare down the file size.

# Note that in general, you should have three to ten times the amount of memory as the size
# of the DataFrame that you want to manipulate. Extra memory should give you enough extra
# space to perform many of the common operations.

In [ ]:
# Reading large CSV files
# Determine how much memory the whole file will take up. We will use the nrows
# parameter of read_csv to limit how much data we load to a small sample:

diamonds = pd.read_csv('../data/diamonds.csv', nrows=1000)
diamonds

In [ ]:
# Use the .info method to see how much memory the sample of data uses:

diamonds.info()

In [ ]:
# We can see that 1,000 rows use about 78.2 KB of memory. If we had 1 billion
# rows, that would take about 78 GB of memory. It turns out that it is possible to rent
# machines in the cloud that have that much memory but let's see if we can take it
# down a little.

In [ ]:
# Use the dtype parameter to read_csv to tell it to use the correct (or smaller) numeric types:

diamonds2 = pd.read_csv('../data/diamonds.csv', nrows=1000,
    dtype={'carat': np.float32, 'depth': np.float32,
           'table': np.float32, 'x': np.float32,
           'y': np.float32, 'z': np.float32,
           'price': np.int16})

In [ ]:
# By changing the numeric types, we use about 62% of the memory.

diamonds2.info()

In [ ]:
# Use the dtype parameter to use change object types to categoricals. First, inspect
# the .value_counts method of the object columns. If they are low cardinality, you
# can convert them to categorical columns to save even more memory:

diamonds.describe()

In [ ]:
# By changing the numeric types, we use about 62% of the memory. Note that we lose
# some precision, which may or may not be acceptable.

diamonds2.describe()

In [ ]:
# Use the dtype parameter to use change object types to categoricals. First, inspect
# the .value_counts method of the object columns. If they are low cardinality, you
# can convert them to categorical columns to save even more memory:

diamonds2.cut.value_counts()

In [ ]:
diamonds2.color.value_counts()

In [ ]:
diamonds2.clarity.value_counts()

In [ ]:
# Because these are of low cardinality, we can convert them to categoricals and use
# around 37% of the original size:

diamonds3 = pd.read_csv('../data/diamonds.csv', nrows=1000,
    dtype={'carat': np.float32, 'depth': np.float32,
           'table': np.float32, 'x': np.float32,
           'y': np.float32, 'z': np.float32,
           'price': np.int16,
           'cut': 'category', 'color': 'category',
           'clarity': 'category'})

In [ ]:
diamonds3.info()

In [ ]:
# If there are columns that we know we can ignore, we can use the usecols
# parameter to specify the columns we want to load. Here, we will ignore columns x, y,
# and z:

In [ ]:
cols = ['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price']
diamonds4 = pd.read_csv('../data/diamonds.csv', nrows=1000,
    dtype={'carat': np.float32, 'depth': np.float32,
           'table': np.float32, 'price': np.int16,
           'cut': 'category', 'color': 'category',
           'clarity': 'category'},
    usecols=cols)

In [ ]:
diamonds4.info()

In [ ]:
cols = ['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price']
diamonds_iter = pd.read_csv('../data/diamonds.csv', nrows=1000,
    dtype={'carat': np.float32, 'depth': np.float32,
           'table': np.float32, 'price': np.int16,
           'cut': 'category', 'color': 'category',
           'clarity': 'category'},
    usecols=cols,
    chunksize=200)

In [ ]:
def process(df):
    return f'processed {df.size} items'

In [ ]:
for chunk in diamonds_iter:
    process(chunk)

In [ ]:
# If the column turns out to be non-numeric, pandas will convert it to an object column, and
# treat the values as strings. String values in pandas take up a bunch of memory as each value
# is stored as a Python string. If we convert these to categoricals, pandas will use much less
# memory as it only stores the string once, rather than creating new strings (even if they repeat)
# for every row.

### How it works\...

### There\'s more \...

In [ ]:
diamonds.price.memory_usage()

In [ ]:
diamonds.price.memory_usage(index=False)

In [ ]:
diamonds.cut.memory_usage()

In [ ]:
diamonds.cut.memory_usage(deep=True)

In [ ]:
diamonds4.to_feather('../tmp/d.arr')
diamonds5 = pd.read_feather('../tmp/d.arr')

In [ ]:
diamonds4.to_parquet('../tmp/d.pqt')

### How to do it\...

In [9]:
import xlwt

In [12]:
beatles.to_excel('../tmp/beat.xlsx')

In [13]:
beatles.to_excel('../tmp/beat.xlsx')

In [14]:
# Read the Excel file with the read_excel function:

beat2 = pd.read_excel('../tmp/beat.xlsx')
beat2

,Unnamed: 0,first,last,birth
0,0,Paul,McCartney,1942
1,1,John,Lennon,1940
2,2,Richard,Starkey,1940
3,3,George,Harrison,1943


In [16]:
# Because this file had an index column included, you can specify that with the index_
# col parameter:

beat2 = pd.read_excel('../tmp/beat.xlsx', index_col=0)
beat2

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


In [17]:
beat2.dtypes

first    object
last     object
birth     int64
dtype: object

### How it works\...

### There\'s more\...

In [19]:
# How to write files to Excel

from openpyxl import Workbook

In [26]:
# We can use pandas to write to a sheet of a spreadsheet. You can pass a sheet_name
# parameter to the .to_excel method to tell it the name of the sheet to create:

xl_writer = pd.ExcelWriter('../tmp/beat.xlsx')
beatles.to_excel(xl_writer, sheet_name='All')
beatles[beatles.birth < 1941].to_excel(xl_writer, sheet_name='1940')


### Working with the ZIP files\...
#### Not clear - Do it again and find some good resource

In [3]:
#  If the CSV file is the only file in the ZIP file, you can just call the read_csv function on it:

autos = pd.read_csv('../data/vehicles.csv.zip', low_memory=False)
autos.head()

,barrels08,barrelsA08,charge120,charge240,city08,...,modifiedOn,startStop,phevCity,phevHwy,phevComb
0,15.695714,0.0,0.0,0.0,19,...,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
1,29.964545,0.0,0.0,0.0,9,...,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
2,12.207778,0.0,0.0,0.0,23,...,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
3,29.964545,0.0,0.0,0.0,10,...,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
4,17.347895,0.0,0.0,0.0,17,...,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0


In [4]:
# So, autos.modifiedOn.dtype is checking the data type of the column named modifiedOn in the autos dataset. 
# The data type could be one of several possibilities, such as integer, float, string, datetime, etc.

autos.modifiedOn.dtype

dtype('O')

In [6]:
# One thing to be aware of is that if you have date columns in the CSV file, they will be
# left as strings. You have two options to convert them. You can use the parse_dates
# parameter from read_csv and convert them when loading the file. Alternatively, you
# can use the more powerful to_datetime function after loading:

In [7]:
autos.modifiedOn

0        Tue Jan 01 00:00:00 EST 2013
1        Tue Jan 01 00:00:00 EST 2013
2        Tue Jan 01 00:00:00 EST 2013
3        Tue Jan 01 00:00:00 EST 2013
4        Tue Jan 01 00:00:00 EST 2013
                     ...             
39096    Tue Jan 01 00:00:00 EST 2013
39097    Tue Jan 01 00:00:00 EST 2013
39098    Tue Jan 01 00:00:00 EST 2013
39099    Tue Jan 01 00:00:00 EST 2013
39100    Tue Jan 01 00:00:00 EST 2013
Name: modifiedOn, Length: 39101, dtype: object

In [11]:
# pd.to_datetime(autos.modifiedOn)  # doctest: +SKIP

In [12]:
# autos = pd.read_csv('../data/vehicles.csv.zip',
#    parse_dates=['modifiedOn'])  # doctest: +SKIP
# autos.modifiedOn

In [13]:
#  If the ZIP file has many files it in, reading a CSV file from it is a little more involved.
# The read_csv function does not have the ability to specify a file inside a ZIP file.
# Instead, we will use the zipfile module from the Python standard library.

import zipfile

In [14]:
with zipfile.ZipFile('../data/kaggle-survey-2018.zip') as z:
    print('\n'.join(z.namelist()))
    kag = pd.read_csv(z.open('multipleChoiceResponses.csv'))
    kag_questions = kag.iloc[0]
    survey = kag.iloc[1:]

multipleChoiceResponses.csv
freeFormResponses.csv
SurveySchema.csv


/tmp/ipykernel_23906/1021377863.py:3: DtypeWarning: Columns (0,2,8,10,21,23,24,25,26,27,28,42,44,54,56,64,81,83,85,87,105,107,109,121,123,125,148,150,157,172,174,192,194,210,218,219,221,223,246,247,249,262,264,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,302,304,306,325,326,329,341,368,371,384,385,389,390,391,393,394) have mixed types. Specify dtype option on import or set low_memory=False.
  kag = pd.read_csv(z.open('multipleChoiceResponses.csv'))


In [15]:
print(survey.head(2).T)

                                                            1          2
Time from Start to Finish (seconds)                       710        434
Q1                                                     Female       Male
Q1_OTHER_TEXT                                              -1         -1
Q2                                                      45-49      30-34
Q3                                   United States of America  Indonesia
...                                                       ...        ...
Q50_Part_5                                                NaN        NaN
Q50_Part_6                                                NaN        NaN
Q50_Part_7                                                NaN        NaN
Q50_Part_8                                                NaN        NaN
Q50_OTHER_TEXT                                             -1         -1

[395 rows x 2 columns]


### Working with databases\...

### SQL Lite Database

In [ ]:
# SQLite database, which is included with Python. However, Python has the ability to connect 
# with most SQL databases and pandas, in turn, can leverage that.

In [21]:
import sqlite3
con = sqlite3.connect('../data/beat.db')
with con:
    cur = con.cursor()
    cur.execute("""DROP TABLE Band""")
    cur.execute("""CREATE TABLE Band(id INTEGER PRIMARY KEY,
        fname TEXT, lname TEXT, birthyear INT)""")
    cur.execute("""INSERT INTO Band VALUES(
        0, 'Paul', 'McCartney', 1942)""")
    cur.execute("""INSERT INTO Band VALUES(
        1, 'John', 'Lennon', 1940)""")
    _ = con.commit()

In [20]:
import sqlalchemy as sa
engine = sa.create_engine(
  'sqlite:///data/beat.db', echo=True)
sa_connection = engine.connect()

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
beat = pd.read_sql('Band', sa_connection, index_col='id')
beat

In [ ]:
sql = '''SELECT fname, birthyear from Band'''
fnames = pd.read_sql(sql, con)
fnames

### Reading JSON 
#### JavaScript Object Notation (JSON)

In [ ]:
# JavaScript Object Notation (JSON) is a common format used for transferring data over the
# internet. Contrary to the name, it does not require JavaScript to read or create. The Python
# standard library ships with the json library that will encode and decode from JSON:

### How it work\'s\...

In [22]:
import json
encoded = json.dumps(people)
encoded

NameError: name 'people' is not defined

In [ ]:
json.loads(encoded)

### How to do it\...

In [ ]:
beatles = pd.read_json(encoded)
beatles

In [ ]:
records = beatles.to_json(orient='records')
records

In [ ]:
pd.read_json(records, orient='records')

In [ ]:
split = beatles.to_json(orient='split')
split

In [ ]:
pd.read_json(split, orient='split')

In [ ]:
index = beatles.to_json(orient='index')
index

In [ ]:
pd.read_json(index, orient='index')

In [ ]:
values = beatles.to_json(orient='values')
values

In [ ]:
pd.read_json(values, orient='values')

In [ ]:
(pd.read_json(values, orient='values')
   .rename(columns=dict(enumerate(['first', 'last', 'birth'])))
)

In [ ]:
table = beatles.to_json(orient='table')
table

In [ ]:
pd.read_json(table, orient='table')

### How it works\...

### There\'s more\...

In [ ]:
output = beat.to_dict()
output

In [ ]:
output['version'] = '0.4.1'
json.dumps(output)

### Reading HTML Tables\...

In [24]:
# Use the read_html function to load all of the tables from: 
# https://en.wikipedia.org/wiki/The_Beatles_discography:

url ='https://en.wikipedia.org/wiki/The_Beatles_discography'
dfs = pd.read_html(url)
len(dfs)

59

In [25]:
dfs[0]

,The Beatles discography,The Beatles discography.1
0,The Beatles in 1965,The Beatles in 1965
1,Studio albums,"12 (UK), 17 (US)"
2,Live albums,5
3,Compilation albums,51
4,Video albums,22
5,Music videos,53
6,EPs,36
7,Singles,63
8,Mash-ups,2
9,Box sets,17


In [26]:
url ='https://en.wikipedia.org/wiki/The_Beatles_discography'
dfs = pd.read_html(url, match='List of studio albums', na_values='—')
len(dfs)

2

In [27]:
dfs[0].columns

MultiIndex([(               'Title',            'Title'),
            (    'Album details[A]', 'Album details[A]'),
            ('Peak chart positions',        'UK [7][8]'),
            ('Peak chart positions',          'AUS [9]'),
            ('Peak chart positions',         'CAN [10]'),
            ('Peak chart positions',         'FRA [11]'),
            ('Peak chart positions',         'GER [12]'),
            ('Peak chart positions',         'NOR [13]'),
            ('Peak chart positions',      'US [14][15]'),
            (      'Certifications',   'Certifications'),
            (               'Sales',            'Sales')],
           )

In [28]:
url ='https://en.wikipedia.org/wiki/The_Beatles_discography'
dfs = pd.read_html(url, match='List of studio albums', na_values='—',
    header=[0,1])
len(dfs)

2

In [29]:
dfs[0]

Title  \
                                                Title   
0                                    Please Please Me   
1                                 With the Beatles[B]   
2                                  A Hard Day's Night   
3                                    Beatles for Sale   
4                                               Help!   
..                                                ...   
8                     The Beatles ("The White Album")   
9                                 Yellow Submarine[C]   
10                                         Abbey Road   
11                                          Let It Be   
12  "—" denotes that the recording did not chart o...   

                                     Album details[A]  \
                                     Album details[A]   
0           Released: 22 March 1963 Label: Parlophone   
1   Released: 22 November 1963 Label: Parlophone (...   
2            Released: 10 July 1964 Label: Parlophone   
3         Released: 4 December 1964 Label: Parlophone   
4           Released: 6 August 1965 Label: Parlophone   
..                                                ...   
8             Released: 22 November 1968 Label: Apple   
9              Released: 17 January 1969 Label: Apple   
10           Released: 26 September 1969 Label: Apple   
11                  Released: 8 May 1970 Label: Apple   
12  "—" denotes that the recording did not chart o...   

                                 Peak chart positions  \
                                            UK [7][8]   
0                                                   1   
1                                                   1   
2                                                   1   
3                                                   1   
4                                                   1   
..                                                ...   
8                                                   1   
9                                                   3   
10                                                  1   
11                                                  1   
12  "—" denotes that the recording did not chart o...   

                                                       \
                                              AUS [9]   
0                                                 NaN   
1                                                 NaN   
2                                                   1   
3                                                   1   
4                                                   1   
..                                                ...   
8                                                   1   
9                                                   4   
10                                                  1   
11                                                  1   
12  "—" denotes that the recording did not chart o...   

                                                       ...  \
                                             CAN [10]  ...   
0                                                 NaN  ...   
1                                                 NaN  ...   
2                                                 NaN  ...   
3                                                 NaN  ...   
4                                                 NaN  ...   
..                                                ...  ...   
8                                                   1  ...   
9                                                   1  ...   
10                                                  1  ...   
11                                                  1  ...   
12  "—" denotes that the recording did not chart o...  ...   

                                                       \
                                             GER [12]   
0                                                   5   
1                                                   1   
2                                            

In [30]:
dfs[0].columns

MultiIndex([(               'Title',            'Title'),
            (    'Album details[A]', 'Album details[A]'),
            ('Peak chart positions',        'UK [7][8]'),
            ('Peak chart positions',          'AUS [9]'),
            ('Peak chart positions',         'CAN [10]'),
            ('Peak chart positions',         'FRA [11]'),
            ('Peak chart positions',         'GER [12]'),
            ('Peak chart positions',         'NOR [13]'),
            ('Peak chart positions',      'US [14][15]'),
            (      'Certifications',   'Certifications'),
            (               'Sales',            'Sales')],
           )

In [31]:
df = dfs[0]
df.columns = ['Title', 'Release', 'UK', 'AUS', 'CAN', 'FRA', 'GER',
    'NOR', 'US', 'Certifications']
df

ValueError: Length mismatch: Expected axis has 11 elements, new values have 10 elements

In [ ]:
res = (df
  .pipe(lambda df_: df_[~df_.Title.str.startswith('Released')])
  .iloc[:-1]
  .assign(release_date=lambda df_: pd.to_datetime(
             df_.Release.str.extract(r'Released: (.*) Label')
               [0]
               .str.replace(r'\[E\]', '')
          ),
          label=lambda df_:df_.Release.str.extract(r'Label: (.*)')
         )
   .loc[:, ['Title', 'UK', 'AUS', 'CAN', 'FRA', 'GER', 'NOR',
            'US', 'release_date', 'label']]
)
res

### How it works\...

### There is more\...

In [ ]:
url = 'https://github.com/mattharrison/datasets/blob/master/data/anscombes.csv'
dfs = pd.read_html(url, attrs={'class': 'csv-data'})
len(dfs)

In [ ]:
dfs[0]